In [1]:
import cv2
import numpy as np
from fastdtw import fastdtw
from scipy.spatial.distance import euclidean

def calculate_optical_flow(video_path):
    cap = cv2.VideoCapture(video_path)
    ret, prev_frame = cap.read()
    prev_gray = cv2.cvtColor(prev_frame, cv2.COLOR_BGR2GRAY)
    flows = []

    while True:
        ret, frame = cap.read()
        if not ret:
            break
        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        flow = cv2.calcOpticalFlowFarneback(prev_gray, gray, None, 0.5, 3, 15, 3, 5, 1.2, 0)
        flows.append(flow)
        prev_gray = gray
    
    cap.release()
    return flows

def compute_mbh(flows):
    mbhs = []

    for flow in flows:
        mbhs = []

    for flow in flows:
        flow_x, flow_y = cv2.split(flow)
        grad_x_x = cv2.Sobel(flow_x, cv2.CV_32F, 1, 0, ksize=5)
        grad_x_y = cv2.Sobel(flow_x, cv2.CV_32F, 0, 1, ksize=5)
        grad_y_x = cv2.Sobel(flow_y, cv2.CV_32F, 1, 0, ksize=5)
        grad_y_y = cv2.Sobel(flow_y, cv2.CV_32F, 0, 1, ksize=5)

        mag_x, angle_x = cv2.cartToPolar(grad_x_x, grad_x_y, angleInDegrees=True)
        mag_y, angle_y = cv2.cartToPolar(grad_y_x, grad_y_y, angleInDegrees=True)

        # Ensure data is in the correct type
        angle_x = np.float32(angle_x)
        angle_y = np.float32(angle_y)

        # Histogram bins and ranges
        bins = [360]
        range_hist = [0, 360]

        hist_x = cv2.calcHist([angle_x], [0], None, bins, range_hist)
        hist_y = cv2.calcHist([angle_y], [0], None, bins, range_hist)

        # Flatten and concatenate histograms
        mbh = np.concatenate((hist_x.flatten(), hist_y.flatten()))
        mbhs.append(mbh)
    
    return np.array(mbhs)


input_video_path = 'dataset/originals/joker.mp4'

input_flows = calculate_optical_flow(input_video_path)
input_mbhs = compute_mbh(input_flows)
print(input_mbhs)



KeyboardInterrupt



In [ ]:
def find_best_match(input_mbhs, query_mbhs):
    distance, path = fastdtw(input_mbhs, query_mbhs, dist=euclidean)
    # Finding the earliest point in the input sequence that aligns with the start of the query
    start_frame = min([input_index for input_index, query_index in path if query_index == 0])
    return start_frame

def find_best_window_match(input_mbhs, query_mbhs):
    min_distance = float('inf')
    best_start = 0
    window_size = len(query_mbhs)
    
    for start in range(len(input_mbhs) - window_size + 1):
        print(start)
        distance, _ = fastdtw(input_mbhs[start:start + window_size], query_mbhs, dist=euclidean)
        if distance < min_distance:
            min_distance = distance
            best_start = start
            
    return best_start

def frame_to_timestamp(frame_index, video_path):
    cap = cv2.VideoCapture(video_path)
    fps = cap.get(cv2.CAP_PROP_FPS)
    cap.release()
    seconds = frame_index / fps
    return f"{int(seconds // 3600):02}:{int((seconds % 3600) // 60):02}:{seconds % 60:.3f}"

query_video_path = 'dataset/queries/joker_query3.mp4'

query_flows = calculate_optical_flow(query_video_path)
query_mbhs = compute_mbh(query_flows)
print(query_mbhs)

starting_frame = find_best_window_match(input_mbhs, query_mbhs)
start_timestamp = frame_to_timestamp(starting_frame, input_video_path)

print(f"Query video starts at frame {starting_frame}, which corresponds to timestamp {start_timestamp}.")